In [206]:
import math
class ReliabilityCalculator:
    def __init__(self, system, n, tr, s, th, rr, ):
        self.mttfn = 1400000        
        self.n = n
        self.system = system
        self.tr = tr
        self.s = s # node total storage capacity
        self.th = th
        self.rr = rr
        
    def initDecStore(self, usage, vdSize = 20000 ):
        self.vdSize = vdSize
        self.processingTime = 0.01 # ms - time to process one vd (decide where to put)

        self.vdsCount = self.n *math.floor(self.s * usage / (100*self.vdSize))
        freeVDsCount = self.n *( math.floor((self.s )/(self.vdSize) ) - self.vdsCount/self.vdsCount) - self.vdsCount# total free vds slots in the system
        self.freeVDsCount = math.floor(freeVDsCount)
        
    def initFullBackup(self, b):
        self.b = b #number of times to visit getPANFBR function
        
    def getVDsInfo(self):
        return (self.vdsCount, self.freeVDsCount)
    
    def getTd(self):
        return self.vdsCount*self.processingTime/((self.n-1)*3600)
        
    def getMTTRn(self): #MTTR node
        if self.system == "decStore":
            mTTRn = self.s /(3600*self.th*self.rr*(self.n-1)) + self.getTd()
        else:
            mTTRn = (self.s /(3600*self.th*self.rr)) + self.tr
        return mTTRn
    
    def getMTTFn(self): #mTTF node
        return self.mttfn
    
    def getPANFBRs(self):
        pANFBER = 1
        if self.system == "raid5":
            pANFBER = self.getMTTRn()/(self.getMTTFn()/(self.n - 1))
        if self.system == "raid10":
            pANFBER = self.getMTTRn()/(self.getMTTFn())
        if self.system == "raid6":
            pANFBER = math.prod([(self.getMTTRn()*(self.n - i))/self.getMTTFn() for i in range (1,3)])#/2
        if self.system == "decStore":
            marc = self.getMARC()
            pANFBER = (((marc*self.getMTTRn())/self.getMTTFn())**marc) * (self.getMTTFn()/math.prod([self.n-i for i in range(1, marc+1)]))
        if self.system == "fullbackup":
            pANFBER = math.prod([(self.getMTTRn()*(self.b+1-1))/self.getMTTFn() for i in range(1, self.b)])
        return pANFBER
    
    def getMTTFs(self): #MTTF system
        MTTFs = ((self.getMTTFn())/self.n)* (1/self.getPANFBRs())
        return MTTFs
    def F(self, n):
        if n == 1:
            return self.freeVDsCount
        return self.F(n-1) - (self.vdsCount/(self.n-n)) - (self.F(n-1)/(self.n-n))
    
    def getMARC(self):
        for i in range (1, self.n):
            if self.F(i) < 0:
                return i-1
    
    def getReliability(self):
        return 100*(1-8760/self.getMTTFs())

    def getNinesCountForNumber(self, number):
        if number == 100:
            return 12
        if str(number).startswith("99"):
            counter = 0
            for i in str(number)[3:]:
                if i == "9":
                    counter +=1
                else:
                    return counter
        return 0
    
    def getAccuracyNines(self, reliabilityArr):
        return [self.getNinesCountForNumber(item) for item in reliabilityArr]

        
        

In [2]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
pANFBER: 74.07407407407408 2000000 5
1799999999.9999995


In [3]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
pANFBER: 74.07407407407408 2000000 5
1799999999.9999995


In [11]:
#raid10
system = ReliabilityCalculator("raid10", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())
print(system.getMTTFn(), system.getMTTRn(), system.getMTTFn())

185.1851851851852
3599999999.999999
2000000 185.1851851851852 2000000


In [15]:
#raid6
system = ReliabilityCalculator("raid6", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())

185.1851851851852
666791999999.9999


## tests start here

In [207]:

#vary by n
reliability = []
n=3
for i in range (10):
    #decStore
    total = 2000000
    vdSize = 100000
    n = n+1
    usage = 70 #%
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getMARC())
print(system.getVDsInfo())

print(system.getAccuracyNines(reliability))

[89.70013310758377, 92.75789848710318, 99.99997057156168, 99.99998410497679, 99.99999046693816, 99.99999384166635, 99.99999999998808, 99.9999999999933, 99.99999999999602, 99.9999999999975]
3
(182, 65)
[0, 0, 4, 4, 5, 5, 10, 11, 11, 11]


In [209]:
#vary by n
reliability = []
n=3
for i in range (10):
    #decStore
    total = 2000000
    n = n+1
    usage = 90 #%
    vdSize = 100000
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getMARC())
print(system.getVDsInfo())
print(system.getAccuracyNines(reliability))


[89.70012074779541, 92.75788762400794, 94.43805018666667, 95.49378457716048, 96.21635045545838, 96.74103183258929, 97.13889925534, 97.45075579507937, 97.7016550145612, 97.90781160725308]
1
(234, 13)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [211]:

#vary by n
reliability = []
n=3
for i in range (10):
    #decStore
    total = 2000000
    n = n+1
    usage = 80 #%
    vdSize = 100000
    system = ReliabilityCalculator("decStore", n, 0, total, 30, 0.5)
    system.initDecStore(usage=usage, vdSize=vdSize)
    reliability.append(system.getReliability())

print(reliability)
print(system.getVDsInfo())
print(system.getAccuracyNines(reliability))


[89.7001269276896, 92.75789305555556, 94.43805519238096, 95.49378930864198, 96.21635499578879, 99.99999384164973, 99.9999957949283, 99.9999970026177, 99.99999778896206, 99.99999832273774]
(208, 39)
[0, 0, 0, 0, 0, 5, 5, 5, 5, 5]


In [212]:
#3-levels backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(3)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[99.99999998423496, 99.99999998029371, 99.99999997635246, 99.99999997241119, 99.99999996846994, 99.99999996452867, 99.9999999605874, 99.99999995664615, 99.99999995270488, 99.99999994876363]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


In [213]:
#2-levels backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(2)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[99.99986757369615, 99.99983446712018, 99.99980136054421, 99.99976825396826, 99.99973514739229, 99.99970204081633, 99.99966893424036, 99.9996358276644, 99.99960272108844, 99.99956961451247]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [214]:
#1-level backup
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("fullbackup", n, 0, total, 30, 0.5)
    system.initFullBackup(1)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))


[97.49714285714286, 96.87142857142858, 96.24571428571429, 95.62, 94.99428571428572, 94.36857142857143, 93.74285714285713, 93.11714285714285, 92.49142857142857, 91.86571428571429]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [216]:
#raid5
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("raid5", n, 0, total, 30, 0.5)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))

[99.99980136054421, 99.99966893424036, 99.99950340136054, 99.99930476190477, 99.99907301587302, 99.99880816326531, 99.99851020408164, 99.998179138322, 99.9978149659864, 99.99741768707483]
[3, 3, 3, 3, 3, 2, 2, 2, 2, 2]


In [217]:
#raid6
reliability = []
total = 2000000
n = 3
for i in range (10):
    n = n+1
    system = ReliabilityCalculator("raid6", n, 0, total, 30, 0.5)
    reliability.append(system.getReliability())

print(reliability)
print(system.getAccuracyNines(reliability))

[99.99999998948998, 99.99999997372494, 99.99999994744988, 99.99999990803728, 99.99999985285967, 99.9999997792895, 99.99999968469928, 99.99999956646151, 99.99999942194867, 99.99999924853327]
[7, 7, 7, 7, 6, 6, 6, 6, 6, 6]
